In [1]:
!pip install transformers

     |████████████████████████████████| 317kB 2.8MB/s 
     |████████████████████████████████| 860kB 30.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=f3e49ef6b52aff7494e084707f1aaf7048490745a32cfdabccc6551138046248
  Stored in directory: /tmp/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import re
import csv
import gensim
from tqdm.auto import tqdm
import tensorflow.keras.layers as L
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import Model, Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.backend import one_hot, clear_session
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer as KerasTokenizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import VotingClassifier
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.naive_bayes import MultinomialNB
from transformers import TFDistilBertModel, DistilBertTokenizer, TFXLNetModel, XLNetTokenizer
import xgboost as xgb
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


/kaggle/input/universal-sentence-encoder/use/use/tfhub_module.pb
/kaggle/input/universal-sentence-encoder/use/use/saved_model.pb
/kaggle/input/universal-sentence-encoder/use/use/variables/variables.index
/kaggle/input/universal-sentence-encoder/use/use/variables/variables.data-00000-of-00001
/kaggle/input/ift3395-ift6390-reddit-comments/data_test.pkl
/kaggle/input/ift3395-ift6390-reddit-comments/sample_submission.csv
/kaggle/input/ift3395-ift6390-reddit-comments/data_train.pkl


In [3]:
submit_predictions = False
train_models = False

In [4]:
import tensorflow as tf2
import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior() # Needed for TensorFlow Hub

In [5]:
datasets = ['train', 'test']
input_path = '/kaggle/input/ift3395-ift6390-reddit-comments/'
data_train, data_test = [np.load(os.path.join(input_path, f'data_{dataset}.pkl'), allow_pickle=True) for dataset in datasets]

In [6]:
def to_dataframe(data):
    if len(data) == 2:
        comment, label = data
        result = pd.DataFrame({'comment': comment, 'label': label})
    else:
        result = pd.DataFrame({'comment': data})
    return result

In [7]:
VAL_FRACTION = 0.1
train_val_df, test_df = (to_dataframe(data) 
                     for data in [data_train, data_test])
train_df, val_df = train_test_split(train_val_df, test_size=VAL_FRACTION)

In [8]:
def vectorize_labels(labels):
    return np.array(labels)[:, np.newaxis]

def to_dense(*args):
    return [item.todense() if hasattr(item, 'todense') else item for item in args]

In [9]:
class USEModel(BaseEstimator, ClassifierMixin):
    def __init__(self, classes, use=None):
        if not use:
            use = hub.Module('../input/universal-sentence-encoder/use/use')
        self.classes = classes
        self.class_count = len(classes)
        self.use = use
        self.one_hot_encoder = OneHotEncoder()
        self.one_hot_encoder.fit(vectorize_labels(classes))
        
    def preprocess(self, data):
        X, y = data
        X_emb = sess.run(self.use(X))
        y_onehot = self.one_hot_encoder.transform(y[:, np.newaxis]).todense()
        return X_emb, y_onehot
    
    def fit(self, x, y):
        self.train_data = self.preprocess((x, y))
        self.idxmap = np.squeeze(
            np.array([np.where(self.one_hot_encoder.categories_[0] == class_label)
                      for class_label in self.classes]))        
        self.build_model(self.train_data[0].shape[1])
        self.train()
        
    def build_model(self, input_shape):
        inputs = layer = L.Input(shape=input_shape, dtype=float)
        layer = L.Dense(2048, activation='relu')(layer)
        layer = L.Dense(2048, activation='relu')(layer)
        layer = L.Dropout(.2)(layer)
        layer = L.Dense(self.class_count, activation='softmax')(inputs)
        self.model = Model(inputs=inputs, outputs=layer)
        self.model.compile(
            loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['acc'])        

    def train(self, epochs=40):
        print("Training")
        X_train, y_train = self.train_data
        self.model.fit(X_train, y_train, epochs=epochs)

    def predict_proba(self, x):
        X_emb = sess.run(self.use(x))        
        probs = self.model.predict(X_emb)
        return probs[:, self.idxmap]

    def predict(self, x):
        probs = self.predict_proba(x)
        idx = np.argmax(probs, axis=1)
        return self.classes[idx]    

In [10]:
class TransformerModel:
    def __init__(self, classes, base=None, max_sequence_length=512, emb_batch_size=100):
        self.classes = classes
        self.class_count = len(classes)
        if not base:
            base = self.get_default_base()
        self.transformer_eff = self.build_transformer_eff()
        self.transformer, self.tokenizer = base            
        self.one_hot_encoder = OneHotEncoder()
        self.one_hot_encoder.fit(vectorize_labels(classes))
        self.max_sequence_length = max_sequence_length
        self.emb_batch_size = emb_batch_size
        self.scaler = StandardScaler()
        self.is_fitted = False
    
    def build_transformer_eff(self):
        @tf2.function
        def eff_model(x):
            outputs = self.transformer(x)
            return tf.reduce_mean(outputs[0], axis=1)        
        return eff_model

    def get_default_base(self):
        model_desc = (TFDistilBertModel, DistilBertTokenizer, 'xlnet-base-cased')
        model_class, tokenizer_class, pretrained_weights = model_desc
        transformer = model_class.from_pretrained(pretrained_weights)
        tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
        return transformer, tokenizer
    
    def compute_embeddings(self, input_ids):
        num_batches = np.ceil(len(input_ids) / self.emb_batch_size)
        batch_iter = tqdm(
            range(0, len(input_ids)+1, self.emb_batch_size),
            total=num_batches, desc='Embeddings')
        embeddings = np.vstack([
            self.transformer_eff(input_ids[n:n+self.emb_batch_size])
            for n in batch_iter])
        if not self.is_fitted:
            self.scaler.fit(embeddings)
            self.is_fitted = True
        return self.scaler.transform(embeddings)
    
    def preprocess(self, data):
        X, y = data
        X_emb = self.preprocess_input(X)
        y_onehot = self.one_hot_encoder.transform(y[:, np.newaxis]).todense()
        return X_emb, y_onehot    
    
    def preprocess_input(self, x):
        input_ids = [
            self.tokenizer.encode(item, add_special_tokens=True) 
            for item in tqdm(x, desc='Tokenization')]
        input_ids = pad_sequences(
            input_ids, maxlen=self.max_sequence_length,
            truncating='post', padding='post')
        return self.compute_embeddings(input_ids)
        
    def build_model(self, input_shape):
        inputs = layer = L.Input(shape=input_shape, dtype=float)
        layer = L.Dense(2048, activation='relu')(layer)
        layer = L.Dense(2048, activation='relu')(layer)
        layer = L.Dropout(.2)(layer)
        layer = L.Dense(self.class_count, activation='softmax')(inputs)
        self.model = Model(inputs=inputs, outputs=layer)
        self.model.compile(
            loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['acc'])        
    
    def train(self, epochs=40, batch_size=128):
        print("Training")
        X_train, y_train = self.train_data
        self.model.fit(X_train, y_train, epochs=epochs)
        
    def fit(self, x, y):
        self.train_data = self.preprocess((x, y))
        self.idxmap = np.squeeze(
            np.array([np.where(self.one_hot_encoder.categories_[0] == class_label)
                      for class_label in self.classes]))        
        self.build_model(self.train_data[0].shape[1])
        self.train()        

    def predict_proba(self, x):
        X_emb = self.preprocess_input(x)
        probs = self.model.predict(X_emb)
        return probs[:, self.idxmap]

    def predict(self, x):
        probs = self.predict_proba(x)
        idx = np.argmax(probs, axis=1)
        return self.classes[idx]

In [11]:
class TfidfNaiveBayesModel(BaseEstimator, ClassifierMixin):
    def __init__(self, classes, alpha=.42):
        self.classes = np.array(classes)
        self.tfidf = TfidfVectorizer()
        self.naive = MultinomialNB(alpha=alpha)        
    def fit(self, x, y):
        vectors = self.tfidf.fit_transform(x)
        self.naive.fit(vectors, y)
        self.idxmap = np.squeeze(
            np.array([np.where(self.naive.classes_ == class_label)
                      for class_label in self.classes]))

    def predict_proba(self, x):
        vectors = self.tfidf.transform(x)
        probabilities = self.naive.predict_proba(vectors)
        return probabilities[:, self.idxmap]
    
    def predict(self, x):
        probs = self.predict_proba(x)
        idx = np.argmax(probs, axis=1)
        return self.classes[idx]

In [12]:
class SimpleVotingClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, estimators, classes):
        self.estimators = estimators
        self.classes = classes
        
    def fit(self, x, y):
        for estimator in self.estimators:
            estimator.fit(x, y)
            
    def predict_proba(self, x):
        probs = np.array(
            [estimator.predict_proba(x) for estimator in self.estimators])
        probs = np.sum(probs, axis=0) / len(self.estimators)
        return probs
    
    def predict(self, x):
        probs = self.predict_proba(x)
        idx = np.argmax(probs, axis=1)
        return self.classes[idx]        

In [13]:
if train_models:
    #TODO: DELETE THIS
    classes = train_val_df.label.unique()
    bert_model = TransformerModel(classes)
    bert_model.fit(train_df.comment, train_df.label)
    predictions = bert_model.predict(val_df.comment)
    accuracy = np.sum(predictions == val_df.label) / len(val_df)
    print(f"Accuracy: {accuracy}")

In [14]:
if train_models:
    sess = tf.InteractiveSession()
    use = hub.Module('../input/universal-sentence-encoder/use/use')
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())

In [15]:
if train_models:
    classes = train_val_df.label.unique()
    use_model = USEModel(classes, use)
    naive_model = TfidfNaiveBayesModel(classes)
    classifiers = [use_model, naive_model]
    voting = SimpleVotingClassifier(estimators=classifiers, classes=classes)

In [16]:
if train_models and not submit_predictions:
    voting.fit(train_df.comment, train_df.label)

In [17]:
if train_models and not submit_predictions:
    predictions = voting.predict(val_df.comment)
    accuracy = np.sum(predictions == val_df.label) / len(val_df)
    print(f"Accuracy: {accuracy}")

In [18]:
if train_models and not submit_predictions:
    predictions = voting.estimators[0].predict(val_df.comment)
    accuracy = np.sum(predictions == val_df.label) / len(val_df)
    print(f"Accuracy: {accuracy}")

In [19]:
if submit_predictions:
    voting.fit(train_val_df.comment, train_val_df.label)
    sanity_check_predictions = voting.predict(val_df.comment)
    accuracy = np.sum(sanity_check_predictions == val_df.label) / len(val_df)
    print(f"Sanity check accuracy: {accuracy} (not a true validation accuracy)")
    
    predictions = voting.predict(data_test)
    with open("predictions.csv", 'w', newline='') as f:
        wr = csv.writer(f)
        wr.writerow(["Id", "Category"])
        for i, prediction in enumerate(predictions):
            wr.writerow((i,prediction))   